In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
np.random.seed(1)
sin_wave = np.array([math.sin(x) for x in np.arange(200)])

X = []
Y = []

seq_len = 10
num_records = len(sin_wave) - seq_len

for i in range(50):
    X.append(sin_wave[i:i+seq_len])

    
X = np.array(X)
X = np.expand_dims(X, axis=2)

Y = np.random.randint(2, size=50)
Y = np.array(Y)
Y = np.expand_dims(Y, axis=1)

X_train =  X[:30]
Y_train =  Y[:30]
X_test = X[30:]
Y_test = Y[30:]


In [ ]:
#RNN Architecture

learning_rate = 0.05
nepoch = 20               
T = seq_len                   # length of sequence
hidden_dim = 10         
output_dim = 1

# bptt_truncate = 0
# min_clip_value = -10
# max_clip_value = 10

In [ ]:
#randomly initialize weights

U = np.random.uniform(0, 1, (hidden_dim, T))
W = np.random.uniform(0, 1, (hidden_dim, hidden_dim))
V = np.random.uniform(0, 1, (output_dim, hidden_dim))
print(U)
print('\n')
print(V)
print('\n')
print(W)

[[0.89460666 0.08504421 0.03905478 0.16983042 0.8781425  0.09834683
  0.42110763 0.95788953 0.53316528 0.69187711]
 [0.31551563 0.68650093 0.83462567 0.01828828 0.75014431 0.98886109
  0.74816565 0.28044399 0.78927933 0.10322601]
 [0.44789353 0.9085955  0.29361415 0.28777534 0.13002857 0.01936696
  0.67883553 0.21162812 0.26554666 0.49157316]
 [0.05336255 0.57411761 0.14672857 0.58930554 0.69975836 0.10233443
  0.41405599 0.69440016 0.41417927 0.04995346]
 [0.53589641 0.66379465 0.51488911 0.94459476 0.58655504 0.90340192
  0.1374747  0.13927635 0.80739129 0.39767684]
 [0.1653542  0.92750858 0.34776586 0.7508121  0.72599799 0.88330609
  0.62367221 0.75094243 0.34889834 0.26992789]
 [0.89588622 0.42809119 0.96484005 0.6634415  0.62169572 0.11474597
  0.94948926 0.44991213 0.57838961 0.4081368 ]
 [0.23702698 0.90337952 0.57367949 0.00287033 0.61714491 0.3266449
  0.5270581  0.8859421  0.35726976 0.90853515]
 [0.62336012 0.01582124 0.92943723 0.69089692 0.99732285 0.17234051
  0.13713575 

In [ ]:
#forward pass
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

for epoch in range(nepoch):
    # check loss on train
    loss = 0.0
    
    # do a forward pass to get prediction
    for i in range(Y.shape[0]):
        x, y = X[i], Y[i]                    # get input, output values of each record
        prev_s = np.zeros((hidden_dim, 1))   # here, prev-s is the value of the previous activation of hidden layer; which is initialized as all zeroes
        for t in range(T):
            new_input = np.zeros(x.shape)    # we then do a forward pass for every timestep in the sequence
            new_input[t] = x[t]              # for this, we define a single input for that timestep
            mulu = np.dot(U, new_input)
            mulw = np.dot(W, prev_s)
            add = mulw + mulu
            s = sigmoid(add)
            mulv = sigmoid(np.dot(V, s))
            prev_s = s

    # calculate error 
        loss_per_record = (y - mulv)**2 / 2
        loss += loss_per_record
    loss = loss / float(y.shape[0])

    # # check loss on val
    # val_loss = 0.0
    # for i in range(Y_val.shape[0]):
    #     x, y = X_val[i], Y_val[i]
    #     prev_s = np.zeros((hidden_dim, 1))
    #     for t in range(T):
    #         new_input = np.zeros(x.shape)
    #         new_input[t] = x[t]
    #         mulu = np.dot(U, new_input)
    #         mulw = np.dot(W, prev_s)
    #         add = mulw + mulu
    #         s = sigmoid(add)
    #         mulv = sigmoid(np.dot(V, s))
    #         prev_s = s

    #     loss_per_record = (y - mulv)**2 / 2
    #     val_loss += loss_per_record
    # val_loss = val_loss / float(y.shape[0])

    print('Epoch: ', epoch + 1, ', Loss: ', loss)

    # train model
    for i in range(Y.shape[0]):
        x, y = X[i], Y[i]
    
        layers = []
        prev_s = np.zeros((hidden_dim, 1))
        dU = np.zeros(U.shape)
        dV = np.zeros(V.shape)
        dW = np.zeros(W.shape)
        
        dU_t = np.zeros(U.shape)
        dV_t = np.zeros(V.shape)
        dW_t = np.zeros(W.shape)
        
        dU_i = np.zeros(U.shape)
        dW_i = np.zeros(W.shape)
        
        # forward pass
        for t in range(T):
            new_input = np.zeros(x.shape)
            new_input[t] = x[t]
            mulu = np.dot(U, new_input)
            mulw = np.dot(W, prev_s)
            add = mulw + mulu
            s = sigmoid(add)
            mulv = sigmoid(np.dot(V, s))
            layers.append({'s':s, 'prev_s':prev_s})
            prev_s = s

        # derivative of pred
        dmulv = (mulv - y)*(1-mulv)*(mulv)
        
        # backward pass
        for t in range(T):
            dV_t = np.dot(dmulv, np.transpose(layers[t]['s']))
            dsv = np.dot(np.transpose(V), dmulv)
            
            ds = dsv
            dadd = add * (1 - add) * ds
            
            dmulw = dadd * np.ones_like(mulw)

            dprev_s = np.dot(np.transpose(W), dmulw)


            for i in range(t, 0, -1):
                ds = dsv + dprev_s
                dadd = add * (1 - add) * ds

                dmulw = dadd * np.ones_like(mulw)
                dmulu = dadd * np.ones_like(mulu)

                dW_i = np.dot(W, layers[t]['prev_s'])
                dprev_s = np.dot(np.transpose(W), dmulw)

                new_input = np.zeros(x.shape)
                new_input[t] = x[t]
                dU_i = np.dot(U, new_input)
                dx = np.dot(np.transpose(U), dmulu)

                dU_t += dU_i
                dW_t += dW_i
                
            dV += dV_t
            dU += dU_t
            dW += dW_t

            # if dU.max() > max_clip_value:
            #     dU[dU > max_clip_value] = max_clip_value
            # if dV.max() > max_clip_value:
            #     dV[dV > max_clip_value] = max_clip_value
            # if dW.max() > max_clip_value:
            #     dW[dW > max_clip_value] = max_clip_value
                
            
            # if dU.min() < min_clip_value:
            #     dU[dU < min_clip_value] = min_clip_value
            # if dV.min() < min_clip_value:
            #     dV[dV < min_clip_value] = min_clip_value
            # if dW.min() < min_clip_value:
            #     dW[dW < min_clip_value] = min_clip_value
        
        # update
        U -= learning_rate * dU
        V -= learning_rate * dV
        W -= learning_rate * dW

Epoch:  1 , Loss:  [[11.47040665]]
Epoch:  2 , Loss:  [[11.47548062]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply


Epoch:  3 , Loss:  [[11.47781683]]
Epoch:  4 , Loss:  [[11.48001257]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: overflow encountered in add


Epoch:  5 , Loss:  [[nan]]
Epoch:  6 , Loss:  [[nan]]
Epoch:  7 , Loss:  [[nan]]
Epoch:  8 , Loss:  [[nan]]
Epoch:  9 , Loss:  [[nan]]
Epoch:  10 , Loss:  [[nan]]
Epoch:  11 , Loss:  [[nan]]
Epoch:  12 , Loss:  [[nan]]
Epoch:  13 , Loss:  [[nan]]
Epoch:  14 , Loss:  [[nan]]
Epoch:  15 , Loss:  [[nan]]
Epoch:  16 , Loss:  [[nan]]
Epoch:  17 , Loss:  [[nan]]
Epoch:  18 , Loss:  [[nan]]
Epoch:  19 , Loss:  [[nan]]
Epoch:  20 , Loss:  [[nan]]


In [ ]:
preds_raw = []
preds = []
for i in range(Y.shape[0]):
    x, y = X[i], Y[i]
    prev_s = np.zeros((hidden_dim, 1))
    # Forward pass
    for t in range(T):
        mulu = np.dot(U, x)
        mulw = np.dot(W, prev_s)
        add = mulw + mulu
        s = sigmoid(add)
        mulv = sigmoid(np.dot(V, s))
        prev_s = s

    preds_raw.append(mulv)
    preds.append(np.round(mulv))
    
preds = np.array(preds)

In [ ]:
preds_raw

[array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]]),
 array([[nan]])]

In [ ]:
result  = pd.DataFrame((Y.reshape(Y.shape[0],).tolist(),preds.reshape(preds.shape[0],).tolist())).T
result.columns  = ("actual", "preds")
conf_matrix  =  pd.crosstab(result.actual, result.preds)
conf_matrix

""
